In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as tck
import sympy as sym
import warnings

from IPython.display import display, Math, clear_output
from numpy.lib.scimath import sqrt as csqrt
from scipy import constants
from scipy.constants import c
from scipy.constants import value
from numpy.random import normal
import os

from optic.core import parameters
from optic.dsp import firFilter, pulseShape,  lowPassFIR,  pnorm
from optic.models import mzm, photodiode
from optic.plot import eyediagram
from commpy.utilities  import signal_power, upsample